## Python 3 code to scrape electricity usage data from ComEd

The following code cell is the preamble to load required packages:

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from datetime import datetime
from selenium.webdriver.chrome.options import Options
import time
import pandas
import shutil
import os
import os.path
import sys

Define the file paths where data will be saved, or loaded from:

In [ ]:
driver_path = 'C:/Users/Mateus/Documents/chromedriver_win32/chromedriver'    # path where Chrome Driver is saved
account_list = 'C:/Users/Mateus/OneDrive/Documentos/AnacondaProjects/ComedScraper_030219/FaultyAccounts1.csv'  # path to csv with utility account numbers
faulty_accts = 'C:/Users/Mateus/OneDrive/Documentos/AnacondaProjects/ComedScraper_030219/FaultyAccounts2.txt'    # where faulty accounts will be stored
results_path = 'C:/Users/Mateus/OneDrive/Documentos/AnacondaProjects/ComedScraper_030219/Results/'    # where results of non-faulty accounts will be stored
download_path = 'C:/Users/Mateus/Downloads/'    # where the Chrome Driver downloads files

Function that defines the actual scraper. This uses ChromeDriver, which should be installed in your system:

In [ ]:
def downCSV( accnNum ):
    url = "https://secure.comed.com/MyAccount/MyService/Pages/UsageDataTool.aspx"
    try:
        options = Options()
#        options.add_argument("--start-minimized")
#        options.add_argument('--headless')
#        options.add_argument('--no-sandbox')
#        driver.set_page_load_timeout(20.0)
        driver = webdriver.Chrome(driver_path, options=options)
        driver.get(url)
        try:
            button = driver.find_element_by_id("ctl00_ctl00_ContentPlaceHolderMain_FeaturedContentZone_ctl00_ctl00_RequestOption_0")
            button.click()
        except:
            print("Request Option Fail")
            driver.quit()
        try:
            textField = driver.find_element_by_id("AccountNumber")
            textField.send_keys(accnNum)
            addButton = driver.find_element_by_id("ctl00_ctl00_ContentPlaceHolderMain_FeaturedContentZone_ctl00_ctl00_Addbtn")
            addButton.click()
        except:
            print("Add Account Fail")
            driver.quit()
        try:
            downloadFile = driver.find_element_by_id("ctl00_ctl00_ContentPlaceHolderMain_FeaturedContentZone_ctl00_ctl00_Downloadbtn")
            downloadFile.click()
        except:
            print("Download File Fail")
            driver.quit()
        time.sleep(1.0)
        driver.quit()
    except:
        print("Load Fail")
        driver.quit()

Define function to grab the account numbers from the source file:

In [ ]:
def grabAccNums():
    fields = ["accountnumber"]
    accNums = pandas.read_csv(account_list, usecols=fields, dtype=str)
    return(accNums)

Code to actually run the scraper, storing results into the specified filepath:

In [ ]:
file = open(faulty_accts, 'a')
accNums = grabAccNums()
for i in range(0, len(accNums)):
    downCSV(accNums.accountnumber[i])
    exists = os.path.isfile(download_path + 'SummaryUsageData.zip')
    if exists:
        stringAccount = str(accNums.accountnumber[i]) + '.zip'
        shareName = results_path + stringAccount
        shutil.move(download_path + 'SummaryUsageData.zip', shareName)
        print(str(accNums.accountnumber[i])+ " SUCCESS")
    else:
        print(str(accNums.accountnumber[i])+ "  FAIL")
        file.write(str(accNums.accountnumber[i]) + "\n")
        file.close()
        file = open(faulty_accts, 'a')
file.close()

In [ ]:
file.close()

In [ ]:
accNums = grabAccNums()
print(accNums.accountnumber[0])